In [ ]:
import pandas as pd
from sklearn import datasets

# Load Iris dataset
iris = datasets.load_iris()

# Create DataFrame from Iris dataset
df = pd.DataFrame(iris.data, columns=iris.feature_names)

# Add target column to DataFrame
df['target'] = iris.target

# Convert target column to float
df['target'] = df['target'].astype(float)

In [ ]:
df.tail()

## Preprocesssing Data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Extract features and target variable from DataFrame
x = df.drop('target', axis=1).values
y = df['target'].values

## NN Architecture

In [ ]:
# Define custom neural network class
class My_NN(nn.Module):
    """
    A custom neural network class.

    Attributes:
        in_feature (int): Number of input features.
        hidden_layers (list): List specifying the number of neurons in each hidden layer.
        out_features (int): Number of output features (or classes).
        activation_function (function): Activation function to be used in hidden layers.
    """

    def __init__(self, in_feature, hidden_layers, out_features, activation_function = F.relu):
        """
        Constructor method for initializing the neural network architecture.

        Args:
            in_feature (int): Number of input features.
            hidden_layers (list): List specifying the number of neurons in each hidden layer.
            out_features (int): Number of output features (or classes).
            activation_function (function, optional): Activation function to be used in hidden layers (default is ReLU).
        """
        super().__init__()
        if len(hidden_layers) < 1:
            raise Exception("My_NN must have at least 1 hidden layer")
        self.layers = []
        self.layers.append(nn.Linear(in_feature, hidden_layers[0]))
        self.add_modules("input_layer", self.layers[0])

        for i in range(1, len(hidden_layers)):
            self.layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            self.add_module(f"hidden_layer_{i}", self.layers[i])

        self.out = nn.Linear(hidden_layers[-1], out_features)

        self.activation_function = activation_function
    
    def forward(self, x):
        """
        Defines the forward pass of the neural network.

        Args:
            x (tensor): Input tensor.

        Returns:
            tensor: Output tensor.
        """
        for i in range(len(self.layers)):
            x = self.activation_function(self.layers[i](x))
        x = self.out(x)
        return x

In [ ]:
# Create instance of the custom neural network
classifier = My_NN(in_feature=4, hidden_layers=[16, 8], out_features=3, activation_function = F.relu)

## NN Train and Eval

### split and format dataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split dataset into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [ ]:
# Convert data to PyTorch tensors datatype
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

### set training parameteres

In [ ]:
# Define loss function
lossFn =  nn.CrossEntropyLoss()

# Define optimizer with classifier parameters and learning rate
optimizer = torch.optimzer.Adam(classifier.parameters(), lr=0.001)

# Define number of epochs
epochs = 2000

### train the model

In [ ]:
# Store the losses in each epoch
losses = []

# Training loop
for i in range(epochs):
    y_pred = classifier.forward(x_train)

    loss = lossFn(y_pred, y_train)

    losses.append(loss.detach().numpy())

    if i % 10 == 0:
        print(f"Epoch {i} - {loss}")

    optimizer.zero_grad() #calculates the differentials that we need
    loss.backward() #output of loss fn, calculate how much correction (how to correct each neuron off given the loss we have)
    optimizer.step() #sends it back through the network, the steps for the optimizer makes one step towards optimizing


## Evaluation

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Plotting the loss over epochs
plt.plot(range(epochs), losses)
plt.ylabel("Loss")
plt.xlabel("Epoch")

In [ ]:
# Evaluate the model on test data
with torch.no_grad():
    y_eval = classifier.forward(x_test)
    loss = lossFn(y_eval, y_test)

loss

In [ ]:
# Plotting the final loss
plt.plot(range(epochs), losses)
plt.plot([epochs], [loss], "g+") # Green cross indicates final loss
plt.ylabel("Loss")
plt.xlabel("Epoch")